# 原始数据加载（docs）

In [1]:
# 实时检测外部函数更新
%load_ext autoreload
%autoreload 2 

In [2]:
import os
# 1. 设置国内镜像源
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from bertopic import BERTopic
import torch # 新增：导入PyTorch（sentence_transformers的底层）
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import joblib
import time
import json
import pandas as pd 
from report_generator import generate_bertopic_report
import re

# Config

In [ ]:
# 从JSON文件加载数据源
with open('data\patents_zf.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 文本
docs_original = [item['combined_text'] for item in data] #文本拼接字段
timestamps = [item['year'] for item in data] # 年份信息

# 版本控制
version = 'V0'
start_time = 2000
end_time = 2025
data_source = 'GF专利'
model_name = "all-mpnet-base-v2" #短文本
# model_name = 'jinaai/jina-embeddings-v2-base-en' #长文本,GPU容易炸

year_range = f'{start_time}-{end_time}'

# 全量数据集的词嵌入结果保存位置，计算耗时长，一定要保存，避免重复计算，只有数据集需要增加或变更时才需要重新计算
embeddings_path = r'results\embedding_results\embeddings_patents_zf_amb_slide_window3.pkl' 
# HDBSCAN网格搜索范围设置
search_sizes = [15, 20, 30, 40, 50, 70, 100,120,150,200]
# 当主题数量非常庞大时，可以进一步层次聚类合并主题，n_groups 是你想要的分组数量
n_groups = 50

# --- UMAP 参数设置 ---
n_neighbors = 15
umap_params = {
    "n_neighbors": n_neighbors,
    "n_components": 5,
    "min_dist": 0.0,
    "metric": 'cosine',
    "random_state": 5,
    "low_memory": True
}

# --- HDBSCAN参数设置 ---
HDBSCAN_cfg = {
    "min_samples": 10,
    "metric": 'euclidean',
    "prediction_data": True
}

# --- vectorizer_model 参数设置 ---
vectorizer_params = {
    "ngram_range": (1, 3), # 词组范围：1-3个词
    "min_df": 20, # 过滤低频噪声，对大样本非常重要
    "max_features": 100000, # 防止内存溢出
    "max_df": 0.7 #过滤掉在超过 50% 文本中都出现的词。这些词通常是行业背景词,会干扰主题关键词的提取。
}

system_prompt = """
你是一名专业的国防专利与技术分析专家，擅长从技术关键词中提炼核心主题方向。

【输入数据格式】
用户将提供一个Python字典：
- 键（key）：主题索引编号（整数，如0, 1, 2...）
- 值（value）：该主题的10个英文关键词字符串，以英文逗号分隔
- 关键词已按重要性降序排列

【核心任务】
为每个主题生成一个中文主题名称，要求：
1. **保守概括**：基于关键词的共同技术内涵，提取"上位技术方向"
2. **权重优先**：主要依据前3-5个高权重关键词，后部关键词仅参考
3. **避免扩展**：不引入新概念，不扩展应用场景，不做过度推断
4. **处理模糊**：若关键词分散，使用更抽象的名称（如"综合技术"）

【主题命名规则】
- 名称应体现国防/军事技术特点
- 使用标准技术术语，避免口语化
- 长度控制在6-15个汉字为宜
- 格式："{主题序号}. {名称}"，序号从1开始连续编号

【输出要求】
- 仅输出合法的JSON对象，无任何额外文本
- JSON结构：{"主题索引": "序号.主题名称"}
- 主题索引与输入保持一致
- 示例输出：{"0": "1.雷达探测", "1": "2.复合材料制备","2": "3.音频处理与降噪"}

【注意事项】
- 关键词可能存在词形变化（单复数等），理解其核心语义
- 国防领域特有术语应保留专业性
"""

任务说明（以文献数据处理为例）：
- 1.将EXCEL数据转换为json格式，按照3TI 、1AB和2KEY WORDS，完成文本的字符串拼接，json文件中需要`year`、`title`、`abstract`、`keywords`以及`combined_text`字段；（AI应该很容易写出来相应的转换代码，该文件作为原始主题建模的原始数据文件）；
- 2.变量`docs`是模型的输入，将json文件中的`combined_text`传入到该向量中形成一个列表型数据；
- 3.根据后续代码跑通这一套BERTopic技术路线

In [ ]:
# 数据清洗
def preprocess_all(texts):
    cleaned_list = []
    for t in texts:
        # 移除特殊字符和多余空格
        t = re.sub(r"[^\w\s\.,;]", " ", t) # 只保留字母、数字、空格和几种标点
        t = re.sub(r"\s+", " ", t).strip() # 移除多余空格
        cleaned_list.append(t)
    return cleaned_list

# 运行
# 检查清洗效果
docs = preprocess_all(docs_original)
# 检查数据对齐
doc_count = len(docs)
time_count = len(timestamps)

# 如果数量对不上就报错，并打印出具体数值
assert doc_count == time_count, f"数据不匹配！文档有 {doc_count} 条，但时间戳有 {time_count} 个。"

print(f"数据检查通过，共 {doc_count} 条记录。")

数据检查通过，共 125510 条记录。


In [5]:
#print('共获取数据总量:', len(docs),'条')

# 创建模型

In [6]:
# CPU同样可以运行，只是速度较慢
# --- 核心：检查并指定设备 ---
# 检查CUDA（即NVIDIA GPU支持）是否可用
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU可用，正在使用: {torch.cuda.get_device_name(0)}")
    # 可选：如果你想使用特定的GPU（如第0块）
    # device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
    print("⚠️  未检测到可用GPU，将使用CPU。")
    import torch
print(torch.__version__) # 查看PyTorch版本
print(torch.cuda.is_available()) # 核心：查看CUDA是否可用

✅ GPU可用，正在使用: NVIDIA GeForce RTX 3050 Laptop GPU
2.6.0+cu124
True


# 预训练模型加载

In [7]:
# 1. 统一定义路径
local_model_path = f"./my_models/{model_name}"  # 固定路径

# 2. 检查并加载模型
if not os.path.exists(local_model_path):
    print(f"模型不存在，正在下载并保存到本地: {model_name}")
    # 直接从huggingface下载amb短文本嵌入模型
    sentence_model = SentenceTransformer(model_name)
    # # 直接从huggingface下载jinai长文本嵌入模型
    # sentence_model = SentenceTransformer(model_name,trust_remote_code=True)

    # 保存到本地
    sentence_model.save(local_model_path)
    print(f"模型已保存至：{local_model_path}")
else:
    print(f"模型已存在，从本地加载: {local_model_path}")
    # 从本地加载
    sentence_model = SentenceTransformer(local_model_path)
    #sentence_model = SentenceTransformer(local_model_path,trust_remote_code=True)
# sentence_model.max_seq_length = 2048

模型已存在，从本地加载: ./my_models/all-mpnet-base-v2


In [ ]:
from slide_window import sliding_window_encode
# 2. 计算并保存词向量
def compute_and_save_embeddings(docs, save_path='embeddings_test'):
    """计算并保存词向量"""
    if os.path.exists(save_path):
        print(f"加载已保存的嵌入: {save_path}")
        with open(save_path, 'rb') as f:
            embeddings = pickle.load(f)
    else:
        print("计算新的嵌入...")
        embeddings = sliding_window_encode(
            docs, 
            sentence_model, 
            window_size=512,
            stride=256,
            inference_batch_size=32
            )



        # embeddings = sentence_model.encode(docs,
        #                                 batch_size=2,
        #                                 show_progress_bar=True,
        #                                 convert_to_numpy=True # 显式转为 numpy 释放 GPU 空间
        #                                 )
        with open(save_path, 'wb') as f:
            pickle.dump(embeddings, f)
        print(f"嵌入已保存到: {save_path}")
    return embeddings

In [9]:
def load_stopwords(file_path):
    """从JSON文件加载停用词列表"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            stopwords = json.load(f)
        print(f"已加载 {len(stopwords['common'])} 个停用词")
        return stopwords['common']  # 转换为集合提高查找效率
    except FileNotFoundError:
        print(f"错误：文件 {file_path} 不存在")
        return set()
    except json.JSONDecodeError:
        print(f"错误：文件 {file_path} 不是有效的JSON格式")
        return set()

# # 测试示例
# stopwords = load_stopwords('data\stopwords.json')
# print(stopwords)

# 📊 BERTopic 核心模型参数设置指南

在 BERTopic 中，**UMAP（降维）** 和 **HDBSCAN（聚类）** 是决定主题质量的两个关键环节。对于 12.5 万条的大规模长文本数据，合理的参数配置能平衡“细节捕捉”与“结构稳健性”。

---

## 一、 UMAP：将高维语义压缩至低维

**主要任务：** 把 BERT 提取的几百维向量降至 5 维，同时保留文本间的相似性。

| 参数名称 | 设置建议 | 通俗理解 |
| --- | --- | --- |
| **`n_neighbors`** | **20** | **视野范围**。决定每个点看多远。值设为 20 能在保留局部细节的同时，兼顾 12.5 万条数据的整体分布，防止主题过于破碎。 |
| **`n_components`** | **5** | **保留维度**。降维后的“维度数”。设置为 5 比 2 或 3 能保留更多语义细节，避免信息在压缩过程中丢失。 |
| **`min_dist`** | **0.0** | **挤压程度**。点与点之间的最小间距。设置为 0.0 会让相似的文档尽可能“抱团”，有助于 HDBSCAN 发现更清晰的边界。 |
| **`metric`** | **'cosine'** | **相似度标准**。文本处理的黄金标准。它关注的是词汇的方向（语义），而不是文本的长短。 |
| **`random_state`** | **5** (或 42) | **结果保险单**。固定随机种子。确保你下次运行代码时，降维后的结果是一模一样的。 |

---

## 二、 HDBSCAN：在低维空间发现主题

**主要任务：** 在降维后的空间里寻找高密度区域，每个高密度区域就是一个“主题”。

| 参数名称 | 设置建议 | 通俗理解 |
| --- | --- | --- |
| **`min_cluster_size`** | **50** | **成团门槛**。定义一个主题最少要包含多少篇文档。对于 12.5 万条数据，50 是一个较细的粒度；如果觉得主题太碎，可以调大至 200-500。 |
| **`min_samples`** | **3** | **严苛程度**。定义核心点的门槛。值越小（如 3），算法越“宽容”，会将更多边缘点纳入主题；值越大，噪声（-1）会越多。 |
| **`metric`** | **'euclidean'** | **测距方式**。由于 UMAP 已经处理过余弦相似度，降维后的 5 维空间通常直接使用欧几里得距离来划分簇。 |

---

## 三、 针对“降噪”与“粒度”的调优秘籍

如果你在运行后发现效果不理想，可以参考以下逻辑进行微调：

### 1. 噪声（-1）比例太高怎么办？

* **方法 A（调松）：** 保持 `min_samples=3` 甚至降为 **1**。这会强制让更多游离的文档归入最近的主题。
* **方法 B（后处理）：** 不动聚类模型，训练后使用 `topic_model.reduce_outliers()` 强制归类。

### 2. 主题太细碎、数量太多怎么办？

* **增加 `min_cluster_size**`：从 50 提高到 **300 或 500**。这是控制主题总数最直接的杠杆。
* **后期合并**：利用 `topic_model.reduce_topics(nr_topics=100)` 手动指定最终想要的主题数。

### 3. 主题之间分不开（语义重叠）？

* **调小 `n_neighbors**`：从 20 降到 **15 或 10**。这会迫使 UMAP 更加关注局部差异，让不同的簇分得更开。

---

> **💡 提示：**
> 对于大规模数据，建议先用这套参数跑一个样本集（如 2 万条），观察主题分布。一旦确定 UMAP 效果不错，请务必**保存 UMAP 结果**（`.npy` 或 `.joblib`），以节省后续反复调参的时间成本。

# BERTopic模型创建

In [10]:
# 你的代码流程：
# +-------------------+       +-------------------------+       +---------------+
# | 你的预计算嵌入     |------>| BERTopic.fit_transform()|------>|最终主题结果   |
# | embeddings_patents|       |                         |       |               |
# | _zf_v1.pkl        |       |                         |       | topics, probs |
# +-------------------+       +-------------------------+       +---------------+
#                                    │
#                                    ▼ 内部处理流程
#                         +-----------------------+
#                         │ 1. 接收你的 embeddings │
#                         +-----------------------+
#                                    │
#                                    ▼
#                         +-----------------------+
#                         │ 2. ✅ UMAP 降维        │
#                         +-----------------------+
#                                    │
#                                    ↓ 
#                         +-----------------------+
#                         │ 3. HDBSCAN 聚类        │
#                         +-----------------------+
#                                    │
#                                    ↓
#                         +----------------------------------+
#                         │ 4. Vectorizer\c-TF-IDF提取主题词等│
#                         +----------------------------------+

## 词向量嵌入

In [11]:
# 计算并保存词向量，全量词向量嵌入，建议保存，以免重复计算
embeddings = compute_and_save_embeddings(docs, embeddings_path)

加载已保存的嵌入: results\embedding_results\embeddings_patents_zf_amb_slide_window3.pkl


## UMAP降维

In [12]:
# 文件路径定义
umap_cache_path = f'umap_model_{data_source}_{start_time}-{end_time}_{version}.joblib'
umap_model = UMAP(**umap_params)
# 检查模型是否存在
if os.path.exists(umap_cache_path):
    print(f"🚀 检测到已存在的降维模型，正在直接加载: {umap_cache_path}")
    umap_embeddings = joblib.load(umap_cache_path)
else:
    print("⏳ 未发现降维模型，开始执行 UMAP 降维计算（大数据量耗时较长）...")
    # 确保 umap_params 已经定义
    umap_embeddings = umap_model.fit_transform(embeddings)
    
    # 保存结果
    joblib.dump(umap_embeddings, umap_cache_path)
    print(f"✔ UMAP 降维完成并已保存至: {umap_cache_path}")

# 接下来可以直接使用 umap_embeddings 进行 HDBSCAN 聚类

🚀 检测到已存在的降维模型，正在直接加载: umap_model_GF专利_2000-2025_V0.joblib


## HDBSCAN聚类（网格搜索）

In [13]:
best_topics = None
best_m_size = None
best_score = float('inf')
search_sizes = [
   15, 20, 30, 40, 50, 70, 100,110,120,130,140,150,160,170,180,190,200,210,220,230,250,300]
search_history = [] # 新增：用于记录网格搜索过程

print(f"{'Size':<10} | {'主题数':<8} | {'负样本数':<10} | {'噪声比例':<10} | {'耗时':<8}")
print("-" * 65)

for m_size in search_sizes:
    start_t = time.time()
  
    clusterer = HDBSCAN(**HDBSCAN_cfg,min_cluster_size=m_size)

    
    labels = clusterer.fit_predict(umap_embeddings)
    
    # 计算指标
    n_outliers = (labels == -1).sum() # 负样本数
    n_topics = len(set(labels)) - (1 if -1 in labels else 0)
    outlier_perc = n_outliers / len(labels)
    duration = time.time() - start_t

    # 保存历史记录
    res = {
        "min_cluster_size": m_size,
        "n_topics": n_topics,
        "n_outliers": n_outliers,
        "outlier_perc": f"{outlier_perc:.1%}",
        "duration": f"{duration:.1f}s",
        "is_best": False
    }
    search_history.append(res)

    # 筛选逻辑：选噪声最小的
    if outlier_perc < best_score:
        best_topics = n_topics
        best_score = outlier_perc
        best_m_size = m_size
    

    print(f"{m_size:<10} | {n_topics:<10} | {n_outliers:<12} | {outlier_perc:<12.1%} | {duration:<8.1f}s")
print("-" * 65)
if best_m_size is None:
    print("未在预设主题数范围内找到参数，建议调低 min_dist 或检查 UMAP 效果")
else:
    print(f"🏆 符合条件的负样本数最少的参数值: min_cluster_size = {best_m_size} (负样本比例: {best_score:.1%},主题数：{best_topics})")

# 标记最优参数
for item in search_history:
    if item["min_cluster_size"] == best_m_size:
        item["is_best"] = True



Size       | 主题数      | 负样本数       | 噪声比例       | 耗时      
-----------------------------------------------------------------
15         | 1120       | 54088        | 43.1%        | 11.7    s
20         | 865        | 53095        | 42.3%        | 6.6     s
30         | 621        | 53239        | 42.4%        | 5.5     s
40         | 475        | 52110        | 41.5%        | 5.6     s
50         | 384        | 51838        | 41.3%        | 4.9     s
70         | 293        | 51908        | 41.4%        | 4.9     s
100        | 225        | 51328        | 40.9%        | 5.5     s
110        | 195        | 48261        | 38.5%        | 4.8     s
120        | 180        | 48440        | 38.6%        | 5.3     s
130        | 168        | 49494        | 39.4%        | 4.9     s
140        | 152        | 49023        | 39.1%        | 5.1     s
150        | 141        | 49114        | 39.1%        | 5.1     s
160        | 136        | 49876        | 39.7%        | 4.9     s
170        | 124 

## 正式创建BERTopic模型

In [14]:
## HDBSCAN聚类
# 使用 ** 进行字典解包
best_clusterer = HDBSCAN(**HDBSCAN_cfg,min_cluster_size=best_m_size)
# 加载停用词
stop_words = load_stopwords('data\stopwords.json')
# 创建CountVectorizer模型,自定义停用词
# vectorizer_model = CountVectorizer(
#     ngram_range = (1,3), # 词组范围：1-3个词
#     stop_words = stop_words, # 停用词
#     min_df = 10, # 过滤低频噪声，对大样本非常重要
#     max_features=100000 # 防止内存溢出
#     )
vectorizer_model = CountVectorizer(
    **vectorizer_params,
    stop_words = stop_words
    )

# 初始化 BERTopic 时带上它
topic_model = BERTopic(
  embedding_model=sentence_model,
  vectorizer_model=vectorizer_model,
  umap_model=umap_model,       # 加上这一行
  hdbscan_model=best_clusterer # 加上这一行
)
#传入训练好的词向量，fit_transformer()包括了UMAP降维+HDBSCAN聚类，不建议这么做，可以将其拆开减少计算
#topics, probs = topic_model.fit_transform(docs, embeddings=embeddings,y = labels) 
#prob计算很花时间
topics, _ = topic_model.fit_transform(docs, embeddings=embeddings)
# 保存整个主题模型


save_path = rf"results\topic_models\bertopic_{data_source}_{year_range}_{version}"
topic_model.save(save_path)
print(rf"主题模型已保存到 results\topic_models\bertopic_{data_source}_{year_range}_{version}")

已加载 175 个停用词


2026-01-28 16:06:14,769 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


主题模型已保存到 results\topic_models\bertopic_GF专利_2000-2025_V0


In [15]:
## 直接加载主题模型
topic_model = BERTopic.load(rf"results\topic_models\bertopic_{data_source}_{year_range}_{version}")
print("初始主题模型加载成功！")

初始主题模型加载成功！


# 原始主题可视化

In [16]:
hierarchical_topics = topic_model.hierarchical_topics(docs)

100%|██████████| 106/106 [00:00<00:00, 165.55it/s]


In [17]:
# 降至二维，再可视化
reduced_embeddings = UMAP(n_neighbors=n_neighbors, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)

In [18]:
import pandas as pd

# --- 0. 基础设置 ---
title_base = f'{year_range}年{data_source}数据Bertopic主题聚类结果一览表-{version}'
output_dir = f"BERTopic_Results_{data_source}{year_range}_Allset_{version}"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- 参数说明 ---
report_name = os.path.join(output_dir, f"report_{year_range}_{data_source}_{version}.html")
generate_bertopic_report(umap_cfg = umap_params, 
                         HDBSCAN_cfg=HDBSCAN_cfg,
                         vectorizer_cfg = vectorizer_params,
                         history= search_history,
                         best_size = best_m_size,
                         model_name = model_name, 
                         output_path = report_name)
print(f"✔ 参数说明文档已生成：{report_name}")

# --- 1. 生成图表 (按照你想要的顺序) ---
#-------------------------------------------------------------------------
# [图 A] 主题关键词条形图 - 存为 barchart.html
fig_barchart = topic_model.visualize_barchart(
    top_n_topics=50, # 只预览前50个主题，若需要所有主题的关键词，则设置为best_topics
    custom_labels=True,
    n_words=10,
    height=400,
    title=f"{year_range}年{data_source}各主题关键词的c-TF-IDF权重得分条形图",
)

fig_barchart.update_layout(
    # 因为每行增加了子图，建议增加总宽度以防重叠
    width=1500, 
    # 统一字体样式
    font=dict(family="KaiTi", size=16),
    title_font=dict(family="KaiTi", size=36, color="black", weight="bold")
)
path_barchart = os.path.join(output_dir, "barchart.html")
fig_barchart.write_html(path_barchart)
#-------------------------------------------------------------------------
# [图 B] 层次聚类图 - 存为 hierarchy.html
fig_hierarchy = topic_model.visualize_hierarchy(
    hierarchical_topics=hierarchical_topics,
    custom_labels=True,
    title=f"{year_range}年{data_source}主题层次聚类图",
    height=800
)
fig_hierarchy.update_layout(
    title_x=0.5,
    # 因为每行增加了子图，建议增加总宽度以防重叠
    width=1500, 
    # 统一字体样式
    font=dict(family="KaiTi", size=16),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_hierarchy = os.path.join(output_dir, "hierarchy.html")
fig_hierarchy.write_html(path_hierarchy)
#-------------------------------------------------------------------------
# [图 C] 文档分布散点图 - 存为 documents.html
fig_documents = topic_model.visualize_documents(
    docs=[doc[:150] + "..." for doc in docs],
    reduced_embeddings=reduced_embeddings,
    custom_labels=True,
    hide_document_hover=False
)
# 更新散点图样式
fig_documents.update_layout(
    title=f"{year_range}年{data_source}主题分布图",
    title_x=0.5,
    width=1500,
    height=1200,
    margin=dict(l=80, r=80, t=100, b=80),
    font=dict(family="KaiTi", size=16, color="black"),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_documents = os.path.join(output_dir, "documents.html")
fig_documents.write_html(path_documents)
#-------------------------------------------------------------------------
# [图 D] 主题时序图 - 存为 hierarchy.html



topics_over_time = topic_model.topics_over_time(docs, timestamps, global_tuning=False, evolution_tuning=False)
fig_topic_time = topic_model.visualize_topics_over_time(
    topics_over_time,
    custom_labels=True)
# 更新散点图样式
fig_topic_time.update_layout(
    title=f"{year_range}年{data_source}主题时序图",
    title_x=0.5,
    width=1500,
    height=800,
    font=dict(family="KaiTi", size=16, color="black"),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_topic_time = os.path.join(output_dir, "topic_overtime_merged.html")
fig_topic_time.write_html(path_topic_time)
# --- 2. 创建导航索引页 (index.html) ---
# 注意：我在这里同步调整了按钮顺序和 iframe 的默认 src
index_content = f"""
<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <title>{title_base}</title>
    <style>
        body {{ font-family: 'Microsoft YaHei', sans-serif; margin: 0; display: flex; flex-direction: column; height: 100vh; background-color: #f4f7f6; }}
        header {{ background: #2c3e50; color: white; padding: 15px 25px; display: flex; justify-content: space-between; align-items: center; box-shadow: 0 2px 5px rgba(0,0,0,0.2); }}
        h1 {{ margin: 0; font-size: 20px; }}
        nav {{ background: #ecf0f1; padding: 10px; display: flex; gap: 10px; border-bottom: 1px solid #ddd; }}
        .nav-btn {{ 
            padding: 8px 15px; background: white; border: 1px solid #bdc3c7; border-radius: 4px; 
            cursor: pointer; text-decoration: none; color: #34495e; font-size: 14px; transition: all 0.3s;
        }}
        .nav-btn:hover {{ background: #3498db; color: white; border-color: #2980b9; }}
        .nav-btn.active {{ background: #3498db; color: white; }}
        #content-frame {{ flex-grow: 1; border: none; width: 100%; }}
    </style>
</head>
<body>
    <header>
        <h1>{title_base}</h1>
        <span style="font-size: 12px; opacity: 0.8;">版本: {version}</span>
    </header>
    
    <nav>
        <a class="nav-btn" href="report_{year_range}_{data_source}_{version}.html" target="chart_frame">✨ {year_range}年{data_source}主题聚类参数说明</a>
        <a class="nav-btn" href="barchart.html" target="chart_frame">📈  {year_range}年{data_source}主题关键词权重图</a>
        <a class="nav-btn" href="hierarchy.html" target="chart_frame">📊  {year_range}年{data_source}主题层次聚类树状图</a>
        <a class="nav-btn" href="documents.html" target="chart_frame">📍  {year_range}年{data_source}主题分布散点图</a>
        <a class="nav-btn" href="topic_overtime_merged.html" target="chart_frame">⌚ {year_range}年{data_source}主题时序图</a>
    
    </nav>

    <iframe name="chart_frame" id="content-frame" src="report_{year_range}_{data_source}_{version}.html"></iframe>

    <script>
        const buttons = document.querySelectorAll('.nav-btn');
        buttons.forEach(btn => {{
            btn.addEventListener('click', function() {{
                buttons.forEach(b => b.classList.remove('active'));
                this.classList.add('active');
            }});
        }});
        // 默认高亮第一个按钮（即参数说明页面）
        buttons[0].classList.add('active');
    </script>
</body>
</html>
"""

with open(os.path.join(output_dir, f"index_{year_range}_{data_source}_res.html"), "w", encoding="utf-8") as f:
    f.write(index_content)

print(f"🎉 结果可视化已完成！已请打开文件夹: {output_dir}查看")

✅ 报告已成功生成至: BERTopic_Results_GF专利2000-2025_Allset_V0\report_2000-2025_GF专利_V0.html
✔ 参数说明文档已生成：BERTopic_Results_GF专利2000-2025_Allset_V0\report_2000-2025_GF专利_V0.html
🎉 结果可视化已完成！已请打开文件夹: BERTopic_Results_GF专利2000-2025_Allset_V0查看


# 层次聚类

# 计算可以进一步合并的主题

In [19]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import fcluster, cut_tree

# 1. 针对数据结构提取 Scipy 链接矩阵
def get_linkage_matrix_final(df):
    # 提取左子节点、右子节点、距离
    # 你的列名是 Child_Left_ID, Child_Right_ID, Distance
    left = df['Child_Left_ID'].values.astype(float)
    right = df['Child_Right_ID'].values.astype(float)
    dist = df['Distance'].values.astype(float)
    
    # 提取主题数量：由于你的 Topics 列存的是列表，我们需要计算每个列表的长度
    num_topics = df['Topics'].apply(len).values.astype(float)
    
    # 合并为 Scipy 需要的 Linkage Matrix (n-1, 4)
    return np.column_stack([left, right, dist, num_topics])

# 2. 执行矩阵转换
# 注意：Scipy 要求矩阵按距离从小到大排序，BERTopic 返回的顺序通常是反的（根节点在最前）
df_sorted = hierarchical_topics.sort_values('Distance')
linkage_matrix = get_linkage_matrix_final(df_sorted)

# 3. 进行分组

clusters = cut_tree(linkage_matrix, n_clusters=n_groups).flatten()

# 4. 映射回原始主题标签
# 获取所有原始主题 ID（排好序，确保与 cut_tree 结果对应）
all_topics = sorted([t for t in topic_model.get_topics().keys() if t != -1])

df_res = pd.DataFrame({
    "topic_id": all_topics,
    "group": clusters
})

# 5. 加上语义标签方便查看
topic_labels = topic_model.generate_topic_labels(nr_words=3, separator="_")
id_to_label = {int(l.split("_")[0]): l for l in topic_labels if not l.startswith("-1")}
df_res['label'] = df_res['topic_id'].map(id_to_label)

# 查看最终结果
print(df_res.head(50))

# 检查每个组包含的主题数
print("\n每组主题数量分布：")
print(df_res['group'].value_counts().sort_index())

    topic_id  group                                              label
0          0      0                          0_fuel_catalyst_fuel cell
1          1      1                     1_microfluidic_analyte_droplet
2          2      2                            2_cache_wireless_packet
3          3      3                 3_waveguide_photonic_optical fiber
4          4      4                    4_turbine_engine_turbine engine
5          5      5                         5_neutron_tomography_gamma
6          6      6                         6_cancer_alkyl_substituted
7          7      7                      7_lithium_battery_electrolyte
8          8      8                         8_tissue_hydrogel_collagen
9          9      9                          9_antibody_antigen_cancer
10        10     10         10_thermoelectric_exchanger_heat exchanger
11        11     11                             11_video_scene_objects
12        12      2                          12_security_key_encrypted
13    

In [20]:
# 1. 按照 group 分组，并将 topic_id 聚合为列表
grouped_topics = df_res.groupby('group')['topic_id'].apply(list).reset_index()

# 2. 格式化输出
print(f"共识别出 {len(grouped_topics)} 个主题组：\n")

for _, row in grouped_topics.iterrows():
    # 将 group 编号加 1 方便阅读（从第一组开始计数）
    group_num = row['group'] + 1
    topic_list = row['topic_id']
    
    # 打印格式：第一组：[0, 1, 2, ...]
    print(f"第{group_num}组：{topic_list}")

共识别出 50 个主题组：

第1组：[0, 91, 103]
第2组：[1, 44]
第3组：[2, 12]
第4组：[3, 90]
第5组：[4, 26]
第6组：[5, 63]
第7组：[6, 19, 62]
第8组：[7, 97]
第9组：[8, 18]
第10组：[9, 40, 64, 75, 98]
第11组：[10, 93]
第12组：[11, 95]
第13组：[13]
第14组：[14, 54]
第15组：[15, 37]
第16组：[16, 106]
第17组：[17, 67, 83, 104]
第18组：[20, 59]
第19组：[21, 80]
第20组：[22, 27, 34, 68, 71, 73, 99]
第21组：[23, 65]
第22组：[24, 29]
第23组：[25, 46, 82]
第24组：[28, 30]
第25组：[31, 35, 52, 102]
第26组：[32, 86]
第27组：[33, 87]
第28组：[36, 105]
第29组：[38, 39, 88]
第30组：[41, 70]
第31组：[42, 100]
第32组：[43, 72, 89]
第33组：[45, 92]
第34组：[47]
第35组：[48, 56]
第36组：[49]
第37组：[50]
第38组：[51, 81, 101]
第39组：[53]
第40组：[55, 66]
第41组：[57, 61]
第42组：[58, 69]
第43组：[60]
第44组：[74]
第45组：[76]
第46组：[77, 85, 94]
第47组：[78]
第48组：[79]
第49组：[84]
第50组：[96]


In [21]:
# 3. 如果你想同时看到这组主题的关键词标签（方便验证语义一致性）
print("\n" + "="*30 + "\n详细语义分组：\n")
for _, row in grouped_topics.iterrows():
    group_num = row['group'] + 1
    # 获取该组下所有主题的标签
    labels = df_res[df_res['group'] == row['group']]['label'].tolist()
    print(f"第{group_num}组 包含主题：")
    for lbl in labels:
        print(f"  - {lbl}")
    print("-" * 20)


详细语义分组：

第1组 包含主题：
  - 0_fuel_catalyst_fuel cell
  - 91_uranium_isotope_isotopes
  - 103_reactor_fuel_uranium
--------------------
第2组 包含主题：
  - 1_microfluidic_analyte_droplet
  - 44_ions_spectrometer_spectrometry
--------------------
第3组 包含主题：
  - 2_cache_wireless_packet
  - 12_security_key_encrypted
--------------------
第4组 包含主题：
  - 3_waveguide_photonic_optical fiber
  - 90_optical fiber_fiber optic_grating
--------------------
第5组 包含主题：
  - 4_turbine_engine_turbine engine
  - 26_fuel_combustion_engine
--------------------
第6组 包含主题：
  - 5_neutron_tomography_gamma
  - 63_coherence_oct_tomography
--------------------
第7组 包含主题：
  - 6_cancer_alkyl_substituted
  - 19_bacterial_infection_bacteria
  - 62_virus_infection_hiv
--------------------
第8组 包含主题：
  - 7_lithium_battery_electrolyte
  - 97_battery_charging_energy storage
--------------------
第9组 包含主题：
  - 8_tissue_hydrogel_collagen
  - 18_nanoparticle_nanoparticles_lipid
--------------------
第10组 包含主题：
  - 9_antibody_antigen_cancer
 

In [22]:
# 1. 提取嵌套列表格式：[[组1序号], [组2序号], ...]
nested_topic_lists = grouped_topics['topic_id'].tolist()

print("--- 嵌套列表输出 ---")
print(nested_topic_lists)
print("\n" + "="*50 + "\n")

--- 嵌套列表输出 ---
[[0, 91, 103], [1, 44], [2, 12], [3, 90], [4, 26], [5, 63], [6, 19, 62], [7, 97], [8, 18], [9, 40, 64, 75, 98], [10, 93], [11, 95], [13], [14, 54], [15, 37], [16, 106], [17, 67, 83, 104], [20, 59], [21, 80], [22, 27, 34, 68, 71, 73, 99], [23, 65], [24, 29], [25, 46, 82], [28, 30], [31, 35, 52, 102], [32, 86], [33, 87], [36, 105], [38, 39, 88], [41, 70], [42, 100], [43, 72, 89], [45, 92], [47], [48, 56], [49], [50], [51, 81, 101], [53], [55, 66], [57, 61], [58, 69], [60], [74], [76], [77, 85, 94], [78], [79], [84], [96]]




In [23]:
# 2. 合并后的主题理解方法一：提取每组的代表性关键词（丢失c-tf-idf权重）
# 逻辑：获取每组内所有主题的关键词，计算词频，取前 10 个作为组关键词
group_keywords = {}

for _, row in grouped_topics.iterrows():
    group_num = row['group']
    topics_in_group = row['topic_id']
    
    all_words_in_group = []
    for t_id in topics_in_group:
        # 获取单个主题的关键词 (返回的是 [(word, score), ...])
        words = [word for word, _ in topic_model.get_topic(t_id)]
        all_words_in_group.extend(words)
    
    # 统计词频并取前 10 个最能代表本组的词
    from collections import Counter
    # 过滤掉一些极其常见的重复词，保留每组独特的语义
    most_common_words = [word for word, count in Counter(all_words_in_group).most_common(10)]
    group_keywords[group_num + 1] = most_common_words

# 3. 格式化展示组关键词
print("--- 各组代表性关键词 ---")
for g_id, keywords in group_keywords.items():
    print(f"第{g_id}组共同特征词: {', '.join(keywords)}")

--- 各组代表性关键词 ---
第1组共同特征词: fuel, reactor, uranium, catalyst, fuel cell, carbon dioxide, biomass, hydrocarbon, porous, anode
第2组共同特征词: analyte, microfluidic, droplet, fluidic, microfluidic device, inlet, microchannel, reservoir, outlet, sampling
第3组共同特征词: cache, wireless, packet, address, instruction, packets, routing, message, request, data processing
第4组共同特征词: optical fiber, grating, waveguide, photonic, optical signal, resonator, cladding, amplifier, semiconductor, coupler
第5组共同特征词: turbine, engine, turbine engine, gas turbine, gas turbine engine, seal, vane, blade, rotor, fan
第6组共同特征词: tomography, imaging system, neutron, gamma, rays, electron beam, computed tomography, radiation detector, detectors, accelerator
第7组共同特征词: alkyl, substituted, optionally substituted, infection, cancer, 6c, aryl, yl, 6c alkyl, inhibitor
第8组共同特征词: lithium, battery, electrochemical, lithium ion, electrochemical cell, electrolyte, cathode, anode, active material, carbonate
第9组共同特征词: tissue, hydrogel, coll

# 合并主题

In [24]:
# 合并主题后,bertopic_model会改变，如需多次合并，重新调用保存的原始模型
topic_model.merge_topics(docs, nested_topic_lists)
#topic_model.get_topic_info()

In [25]:
# 获取所有主题的字典 {topic_id: [(word, score), ...]}
all_topics_dict = topic_model.get_topics()
# 过滤掉 -1 (噪声)，并将每个词列表转为逗号连接的字符串
clean_topics_dict = {
    topic_id: ",".join([word for word, score in words_list])
    for topic_id, words_list in all_topics_dict.items()
    if topic_id != -1
}

for item in clean_topics_dict.items():
    print(item)
    
# 检查结果
print(f'已获取{len(clean_topics_dict)}个主题的详细信息，现在通过AI判断主题名称……') 

(0, 'microfluidic,analyte,droplet,fluidic,microchannel,spectrometer,droplets,analytes,reagent,culture')
(1, 'catalyst,fuel cell,reactor,biomass,feedstock,catalytic,hydrocarbons,ammonia,uranium,exhaust')
(2, 'packet,message,request,packets,routing,server,data processing,communication system,file,encrypted')
(3, 'expression,disease,protein,cancer,gene,administering,peptide,patient,inhibitor,tumor')
(4, 'alkyl,6c,cancer,optionally substituted,aryl,yl,infection,inhibitor,heteroaryl,ch')
(5, 'photonic,optical fiber,optical signal,resonator,cladding,grating,coupler,refractive,waveguides,refractive index')
(6, 'turbine,combustion,rotor,nozzle,fan,compressor,shaft,exhaust,injector,liner')
(7, 'antibody,antigen,protein,amino,amino acid,fragment,cancer,receptor,polypeptide,tumor')
(8, 'hydrogel,scaffold,collagen,biocompatible,bone,therapeutic,biodegradable,lipid,hydrophobic,graft')
(9, 'neutron,tomography,gamma,rays,coherence,electron beam,computed tomography,imaging system,accelerator,ct')
(10,

In [26]:
print(clean_topics_dict)

{0: 'microfluidic,analyte,droplet,fluidic,microchannel,spectrometer,droplets,analytes,reagent,culture', 1: 'catalyst,fuel cell,reactor,biomass,feedstock,catalytic,hydrocarbons,ammonia,uranium,exhaust', 2: 'packet,message,request,packets,routing,server,data processing,communication system,file,encrypted', 3: 'expression,disease,protein,cancer,gene,administering,peptide,patient,inhibitor,tumor', 4: 'alkyl,6c,cancer,optionally substituted,aryl,yl,infection,inhibitor,heteroaryl,ch', 5: 'photonic,optical fiber,optical signal,resonator,cladding,grating,coupler,refractive,waveguides,refractive index', 6: 'turbine,combustion,rotor,nozzle,fan,compressor,shaft,exhaust,injector,liner', 7: 'antibody,antigen,protein,amino,amino acid,fragment,cancer,receptor,polypeptide,tumor', 8: 'hydrogel,scaffold,collagen,biocompatible,bone,therapeutic,biodegradable,lipid,hydrophobic,graft', 9: 'neutron,tomography,gamma,rays,coherence,electron beam,computed tomography,imaging system,accelerator,ct', 10: 'lithium 

# AI主题识别

In [27]:
# AI识别
user_prompt = f'''
请分析以下主題关键词字典，
为每个主题生成对应的中文主题名称，
并返回 JSON 字典：\n{clean_topics_dict}
'''

In [28]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import re
# 加载 .env 文件
load_dotenv(".env")
API_KEY=os.environ.get('DEEPSEEK_API_KEY')
deepseek_chat_model = "deepseek-chat" # DeepSeek-V3.2的非思考模式
client = OpenAI(
    api_key=API_KEY,
    base_url="https://api.deepseek.com")
# AI识别智能体参数设置：创建专门用于地址推理的 LLM 实例
Deepseek_reasponse = client.chat.completions.create(
    model=deepseek_chat_model,
    messages=[ # 对话消息列表
        {"role": "system", "content": system_prompt}, # 系统提示词，定义助手的行为
        {"role": "user", "content": user_prompt},
    ],
    response_format={'type': 'json_object'}, #强制json格式返回
    stream=False # 非流式响应（一次性返回完整结果）
)
content = json.loads(Deepseek_reasponse.choices[0].message.content)
#content = re.sub(r"^```json\s*|\s*```$", "", content.strip())
# 将键转为整型
formatted_labels = {int(k): v for k, v in content.items()}
print(formatted_labels)
# 使用自定义标签（需要先设置）
topic_model.set_topic_labels(formatted_labels)

{0: '1. 微流控分析技术', 1: '2. 催化剂与反应器技术', 2: '3. 数据包通信与处理', 3: '4. 疾病基因与蛋白质表达', 4: '5. 有机化合物抑制剂', 5: '6. 光子学与光波导技术', 6: '7. 涡轮发动机技术', 7: '8. 抗体与抗原蛋白质', 8: '9. 生物材料与组织工程', 9: '10. 中子与伽马射线成像', 10: '11. 锂离子电池技术', 11: '12. 病毒与核酸技术', 12: '13. 光学显微成像系统', 13: '14. 雷达与导航定位', 14: '15. 催化剂与聚合物合成', 15: '16. 基因表达与酶技术', 16: '17. 机械结构与防护', 17: '18. 文档与图像数据处理', 18: '19. 热电与热交换技术', 19: '20. 图像场景与监视', 20: '21. 患者外科手术技术', 21: '22. 光伏与光电探测器', 22: '23. 核酸测序与杂交', 23: '24. 爆炸物与武器系统', 24: '25. 神经刺激与植入技术', 25: '26. 磁共振成像技术', 26: '27. 氮化镓发光器件', 27: '28. 结构健康监测技术', 28: '29. 半导体场效应晶体管', 29: '30. 量子与超导技术', 30: '31. 旋翼与电动推进', 31: '32. 疼痛与受体疾病', 32: '33. 无人系统与轨迹规划', 33: '34. 微电子制造与封装', 34: '35. 电力系统与故障处理', 35: '36. 高温合金与复合材料', 36: '37. 纳米线制造技术', 37: '38. 水下无人航行器', 38: '39. 磁随机存取存储器', 39: '40. 超声成像与治疗', 40: '41. MEMS谐振器技术', 41: '42. 增材制造技术', 42: '43. 碳纳米管技术', 43: '44. 音频处理与降噪', 44: '45. 钻井与地质勘探', 45: '46. 碳纤维复合材料', 46: '47. 平板显示器件', 47: '48. 石墨烯与纳米复合材料', 48: '49. 癌症基因表达分析', 49: '50. 能量收集与转换'}


In [29]:
import pandas as pd
from topicvalue import export_topics_over_time_html

h_data = {
        "n_groups": n_groups,
        "all_topics": all_topics # 原始主题列表
    }

# --- 0. 基础设置 ---
title_base = f'{year_range}年{data_source}进一步层次聚类结果一览表-{version}'
output_dir = f"BERTopic_Results_{data_source}{year_range}_Allset_merged{n_groups}_{version}"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- 参数说明 ---
report_name = os.path.join(output_dir, f"report_{year_range}_{data_source}_{version}.html")
generate_bertopic_report(umap_cfg = umap_params, 
                         HDBSCAN_cfg=HDBSCAN_cfg,
                         vectorizer_cfg = vectorizer_params,
                         history= search_history,
                         best_size = best_m_size,
                         model_name = model_name, 
                         output_path = report_name,
                         hierarchical_data = h_data)
# print(f"✔ 参数说明文档已生成：{report_name}")

# --- 1. 生成图表 (按照你想要的顺序) ---
#-------------------------------------------------------------------------
# [图 A] 主题关键词条形图 - 存为 barchart.html
fig_barchart = topic_model.visualize_barchart(
    top_n_topics=len(formatted_labels),
    custom_labels=True,
    n_words=10,
    height=400,
    title=f"{year_range}年{data_source}各主题关键词的c-TF-IDF权重得分条形图",
)

fig_barchart.update_layout(
    # 因为每行增加了子图，建议增加总宽度以防重叠
    width=1500, 
    # 统一字体样式
    font=dict(family="KaiTi", size=16),
    title_font=dict(family="KaiTi", size=36, color="black", weight="bold")
)
path_barchart = os.path.join(output_dir, "barchart.html")
fig_barchart.write_html(path_barchart)
#-------------------------------------------------------------------------
# [图 B] 层次聚类图 - 存为 hierarchy.html
fig_hierarchy = topic_model.visualize_hierarchy(
    hierarchical_topics=hierarchical_topics,
    custom_labels=True,
    title=f"{year_range}年{data_source}主题层次聚类图",
    height=800
)
fig_hierarchy.update_layout(
    title_x=0.5,
    # 因为每行增加了子图，建议增加总宽度以防重叠
    width=1500, 
    # 统一字体样式
    font=dict(family="KaiTi", size=16),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_hierarchy = os.path.join(output_dir, "hierarchy.html")
fig_hierarchy.write_html(path_hierarchy)
#-------------------------------------------------------------------------
# [图 C] 文档分布散点图 - 存为 documents.html
fig_documents = topic_model.visualize_documents(
    docs=[doc[:150] + "..." for doc in docs],
    reduced_embeddings=reduced_embeddings,
    custom_labels=True,
    hide_document_hover=False
)
# 更新散点图样式
fig_documents.update_layout(
    title=f"{year_range}年{data_source}主题分布图",
    title_x=0.5,
    width=1500,
    height=1200,
    margin=dict(l=50, r=250, t=100, b=50),
    font=dict(family="KaiTi", size=16, color="black"),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_documents = os.path.join(output_dir, "documents.html")
fig_documents.write_html(path_documents)
#-------------------------------------------------------------------------
# [图 D] 主题时序图 - 存为 topic_overtime_merged.html

topics_over_time = topic_model.topics_over_time(docs, timestamps, global_tuning=False, evolution_tuning=False)
fig_topic_time = topic_model.visualize_topics_over_time(
    topics_over_time,
    custom_labels=True)
# 更新散点图样式
fig_topic_time.update_layout(
    title=f"{year_range}年{data_source}主题时序图",
    title_x=0.5,
    width=1500,
    height=800,
    font=dict(family="KaiTi", size=16, color="black"),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_topic_time = os.path.join(output_dir, "topic_overtime_merged.html")
fig_topic_time.write_html(path_topic_time)

#-------------------------------------------------------------------------
# [图E] 主题年份矩阵图 - 存为 topic_overtime_merged.html
path_topic_time_value = os.path.join(output_dir, "topics_over_time_value.html")
export_topics_over_time_html(
    topics_over_time=topics_over_time,
    topic_labels=formatted_labels,
    output_html=path_topic_time_value,
    remove_outliers=False
)


# --- 2. 创建导航索引页 (index.html) ---
# 注意：我在这里同步调整了按钮顺序和 iframe 的默认 src
index_content = f"""
<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <title>{title_base}</title>
    <style>
        body {{ font-family: 'Microsoft YaHei', sans-serif; margin: 0; display: flex; flex-direction: column; height: 100vh; background-color: #f4f7f6; }}
        header {{ background: #2c3e50; color: white; padding: 15px 25px; display: flex; justify-content: space-between; align-items: center; box-shadow: 0 2px 5px rgba(0,0,0,0.2); }}
        h1 {{ margin: 0; font-size: 20px; }}
        nav {{ background: #ecf0f1; padding: 10px; display: flex; gap: 10px; border-bottom: 1px solid #ddd; }}
        .nav-btn {{ 
            padding: 8px 15px; background: white; border: 1px solid #bdc3c7; border-radius: 4px; 
            cursor: pointer; text-decoration: none; color: #34495e; font-size: 14px; transition: all 0.3s;
        }}
        .nav-btn:hover {{ background: #3498db; color: white; border-color: #2980b9; }}
        .nav-btn.active {{ background: #3498db; color: white; }}
        #content-frame {{ flex-grow: 1; border: none; width: 100%; }}
    </style>
</head>
<body>
    <header>
        <h1>{title_base}</h1>
        <span style="font-size: 12px; opacity: 0.8;">版本: {version}</span>
    </header>
    
    <nav>
        <a class="nav-btn" href="report_{year_range}_{data_source}_{version}.html" target="chart_frame">✨ {year_range}年{data_source}主题聚类参数说明</a>
        <a class="nav-btn" href="barchart.html" target="chart_frame">📈  {year_range}年{data_source}主题关键词权重图</a>
        <a class="nav-btn" href="hierarchy.html" target="chart_frame">📊  {year_range}年{data_source}主题层次聚类树状图</a>
        <a class="nav-btn" href="documents.html" target="chart_frame">📍  {year_range}年{data_source}主题分布散点图</a>
        <a class="nav-btn" href="topic_overtime_merged.html" target="chart_frame">⌚ {year_range}年{data_source}主题时序图</a>
        <a class="nav-btn" href="topics_over_time_value.html" target="chart_frame">⌚ {year_range}年{data_source}主题年份矩阵</a>
    
    </nav>

    <iframe name="chart_frame" id="content-frame" src="report_{year_range}_{data_source}_{version}.html"></iframe>

    <script>
        const buttons = document.querySelectorAll('.nav-btn');
        buttons.forEach(btn => {{
            btn.addEventListener('click', function() {{
                buttons.forEach(b => b.classList.remove('active'));
                this.classList.add('active');
            }});
        }});
        // 默认高亮第一个按钮（即参数说明页面）
        buttons[0].classList.add('active');
    </script>
</body>
</html>
"""

with open(os.path.join(output_dir, f"index_{year_range}_{data_source}_res.html"), "w", encoding="utf-8") as f:
    f.write(index_content)

print(f"🎉 结果可视化已完成！已请打开文件夹: {output_dir}查看")

✅ 报告已成功生成至: BERTopic_Results_GF专利2000-2025_Allset_merged50_V0\report_2000-2025_GF专利_V0.html
[OK] HTML 已生成：BERTopic_Results_GF专利2000-2025_Allset_merged50_V0\topics_over_time_value.html
🎉 结果可视化已完成！已请打开文件夹: BERTopic_Results_GF专利2000-2025_Allset_merged50_V0查看
